# MATH&ML-8 Алгоритмы на основе деревьев решений.
###  Содержание <a class="anchor" id=0></a>

- [1. Введение](#1)
- [2. Деревья решений (регрессия и классификация)](#2)
- [3. Алгоритм CART. Параметры](#3)
- [4. Алгоритм CART. Неоднородность](#4)
- [5. Алгоритм CART. Рекурсия](#5)
- [6. Алгоритм CART. Значимость](#6)
- [7. Деревья решений. Практика](#7)
- [8. Итоги](#8)

# 1. Введение <a class="anchor" id=1></a>

[к содержанию](#0)

# 2. Деревья решений (регрессия и классификация) <a class="anchor" id=2></a>

[к содержанию](#0)

# 3. Алгоритм CART. Параметры <a class="anchor" id=3></a>

[к содержанию](#0)

# 4. Алгоритм CART. Неоднородность <a class="anchor" id=4></a>

[к содержанию](#0)

# 5. Алгоритм CART. Рекурсия <a class="anchor" id=5></a>

[к содержанию](#0)

# 6. Алгоритм CART. Значимость <a class="anchor" id=6></a>

[к содержанию](#0)

# 7. Деревья решений. Практика <a class="anchor" id=7></a>

[к содержанию](#0)

# 8. Итоги <a class="anchor" id=8></a>

[к содержанию](#0)